In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow import expand_dims
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import time

In [ ]:
# To Do
# Run the model on the sample data for 25 epochs (no change to dtype)
# Run the model after changing the dtype to a lower byte size (to see if speed improves)
# Find optimal dtypet
# Check that lowering dtype has no consequence

In [8]:
datagen = ImageDataGenerator(
    rescale = 1./255, # need to remember we have normalised here so don't do it within neural networks
    rotation_range = 5,
    zoom_range = 0.1,
    brightness_range = (0.95, 0.95),
    horizontal_flip = True,
    vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.2,
    dtype = tf.float32
)

In [10]:
train_generator = datagen.flow_from_directory(
    '../raw_data/SampleData/train',
    target_size = (224, 224),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 123
)

Found 400 images belonging to 4 classes.


In [11]:
validation_generator = datagen.flow_from_directory(
    '../raw_data/SampleData/train',
    target_size = (224, 224),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation',
    seed = 123
)

Found 100 images belonging to 4 classes.


In [12]:
# The model architecture

base_model = VGG19(include_top=True, input_shape=(224, 224, 3), weights='imagenet')
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='AUC')

es = EarlyStopping(patience=5, restore_best_weights=True)

2021-09-01 12:50:58.079990: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
%%time

model.fit(train_generator, validation_data=validation_generator, epochs=25, callbacks=[es], verbose=1)

2021-09-01 12:51:12.129279: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
 2/13 [===>..........................] - ETA: 23:06 - loss: 1.3872 - auc: 0.5182

KeyboardInterrupt: 

# Messing around with ways to get samples from data

In [ ]:
pathways = []
counter = 0
for filename in train_generator.filenames:
    pathways.append('/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train/'+filename)
    counter += 1
    if counter > 1:
        break
print(pathways)
len(pathways)